<a href="https://colab.research.google.com/github/KendallScott/ML2/blob/main/Homework_4_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework 4: Transfer Learning
Below you see a tutorial from Keras on using transfer learning. They train their models on half  the MNIST dataset digits (i.e. digits 0-4) and “transfer” the model to the second half (i.e, digits 5-9).



Your homework is to train on all digits and make your own handwritten data set of the 5 characters {A, B, C, D, E}  and “transfer” your MNIST trained model over to the dataset you created. In other words, train on the full MNIST datasets (i.e. digits 0-9) and transfer on the {A, B, C, D, E} image dataset.


10 examples of each character

Please do not use any other data resources from the web such as the emnist dataset. Figuring out the challenges of making your own handwritten character dataset is part of the exercise for this homework!   



Submission instructions: Include the images you generated as part of your submission. Include any code used for the character image preprocessing. In a README file include details on your data generation process.



Submit only .py/.ipynb and README files.

Code source: https://github.com/awslabs/keras-apache-mxnet/blob/master/examples/mnist_transfer_cnn.py

In [238]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [162]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [163]:
from __future__ import print_function
import datetime
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 221, 229
filters = 32
pool_size = 2
kernel_size = 3
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [147]:
from __future__ import print_function
import datetime
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3


In [277]:
def train_model(model, train, test, num_classes, epochs):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])


In [274]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]
x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5


In [275]:
x_test_gte5.shape

(4861, 28, 28)

In [276]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

now = datetime.datetime.now

batch_size = 128
num_classes = 5
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)



In [278]:

# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [279]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

# create complete model
model = Sequential(feature_layers + classification_layers)


In [280]:

# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes, 5)

# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes, 5)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 54s 220ms/step - loss: 1.6060 - accuracy: 0.2025 - val_loss: 1.5792 - val_accuracy: 0.3071
Epoch 2/5
240/240 [==============================] - 51s 213ms/step - loss: 1.5682 - accuracy: 0.3257 - val_loss: 1.5370 - val_accuracy: 0.6503
Epoch 3/5
240/240 [==============================] - 50s 207ms/step - loss: 1.5290 - accuracy: 0.4458 - val_loss: 1.4913 - val_accuracy: 0.7970
Epoch 4/5
240/240 [==============================] - 49s 206ms/step - loss: 1.4850 - accuracy: 0.5448 - val_loss: 1.4401 - val_accuracy: 0.8544
Epoch 5/5
240/240 [==============================] - 50s 208ms/step - loss: 1.4337 - accuracy: 0.6211 - val_loss: 1.3796 - val_accuracy: 0.8766
Training time: 0:04:23.003143
Test score: 1.3796383142471313
Test accuracy: 0.8766297101974487
x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [======================

In [360]:
import os
import numpy as np
import cv2
import pandas as pd
from PIL import Image

directory_path = '/content/drive/MyDrive/handwriting/'
directory_files = os.listdir(directory_path)
data = []
label = []

for x,i in enumerate(directory_files):
    # load the image, pre-process it, and store it in the data list
    img_array = cv2.imread(os.path.join(directory_path, i), cv2.IMREAD_GRAYSCALE)
    img_array = cv2.resize(img_array, (28, 28))


    i = i[0:1]
    i = int(i)-1
    label.append(i)

    img_pil = Image.fromarray(img_array)
    img_pil = np.array(img_pil)

    img_28x28  = img_pil.reshape(-1,1).T

    img_28x28 = np.array(img_28x28.reshape((28, 28)))

    data.append(img_28x28)



In [361]:
data[0].shape

(28, 28)

In [362]:

data = np.array(data, dtype=np.uint8)
label = np.array(label, dtype=np.uint8)

In [363]:
x_train_drawn, x_test_drawn, y_train_drawn, y_test_drawn = train_test_split(data, label, test_size=0.3,  random_state=0, stratify=label)


In [ ]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (x_train_drawn, y_train_drawn),
            (x_test_drawn, y_test_drawn), num_classes, 50)

x_train shape: (34, 28, 28, 1)
34 train samples
15 test samples
Epoch 1/50
1/1 [==============================] - ETA: 0s - loss: 1.6273 - accuracy: 0.2647